This notebook is meant to be an initial setup of an AI agent in AWS Bedrock. This allows for code interpretation and passing in files, which makes it ideal for an autonomous data analyst. We use Claude 3.5 Sonnet as the foundation model. This agent will be used in the user interface of the mapping application.

Based on AWS example: https://github.com/build-on-aws/agents-for-amazon-bedrock-sample-feature-notebooks/blob/main/notebooks/preview-agent-code-interpreter.ipynb

In [1]:
import boto3
import json
import time
import csv
import io

import matplotlib.pyplot as plt


## 1) Setup

This first code block is to be run once to initialize the agent on AWS Bedrock. This includes the appropriate IAM role and policy and instruction set for agent. We choose us-west-2 because that is the region that claude 3.5 is available to use code interpreter.

In [ ]:
REGION_NAME = 'us-west-2'
AGENT_NAME = 'climate-risk-map-ai-agent'
FOUNDATIONAL_MODEL = 'anthropic.claude-3-5-sonnet-20241022-v2:0'
IAM_ROLE_NAME = f'{AGENT_NAME}-role'
POLICY_NAME = f'{AGENT_NAME}-policy'
TRUST_POLICY = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Principal": {
                "Service": "bedrock.amazonaws.com"
            },
            "Action": "sts:AssumeRole"
        }
    ]
}
POLICY = {
    "Version": "2012-10-17",
    "Statement": [
        {
            "Effect": "Allow",
            "Action": [
                "bedrock:InvokeModel"
            ],
            "Resource": [
                f"arn:aws:bedrock:{REGION_NAME}::foundation-model/{FOUNDATIONAL_MODEL}"
            ]
        }
    ]
}
INSTRUCTION = """"
You are an advanced AI agent with capabilities in code execution, chart generation, and complex data analysis. 
Your primary function is to assist users by solving problems and fulfilling requests through these capabilities. 
Here are your key attributes and instructions:

Domain and Data:
You are an agent in a climate risk application. This application is an interactive map
that displays power grid features and various climate hazard overlays. The user can select
a boundary box on the map and then invoke you. You will be passed a clean CSV file where each
record is a given power grid feature. Here is a table of labels for column osm_subtype and what
features they represent. 

osm_subtype == line : Power Transmission Line
osm_subtype == plant : Power Plant
osm_subtype == minor_line : Power Distribution Line
osm_subtype == substation : Power Substation

The osm_id column contains the unique OpenStreetMap ID of the given power grid feature. This may be duplicated in the dataset
due to certain feature being located with multiple counties, shown in the county or city column.

There will be column labels that start with "ensemble_". This represents the climate model ensemble.
The values of these columns are the canadian fire weather index values for the given year, month, and SSP values.
This is a proxy for fire risk at each asset, specifically the atmospheric conditions conducive for for fires starting
and spreading.

Code Execution:

You have access to a Python environment where you can write and execute code in real-time.
When asked to perform calculations or data manipulations, always use this code execution capability to ensure accuracy.
After executing code, report the exact output and explain the results.

Data Analysis:
Using the given file information, your data analaysis will be geared towards a summary of the power grid assets in the
selected area, a description of what is most at risk and least at risk based on ensemble values, and potential insights
given the numerical columns in the data. 

Problem-Solving Approach:

When presented with a problem or request, break it down into steps.
Clearly communicate your thought process and the steps you're taking.
If a task requires multiple steps or tools, outline your approach before beginning.

"""


bedrock_agent = boto3.client(service_name = 'bedrock-agent', region_name = REGION_NAME)
iam = boto3.client('iam')

print("Creating the IAM policy and role...")

# Create IAM role and attach policy
role = iam.create_role(
    RoleName=IAM_ROLE_NAME,
    AssumeRolePolicyDocument = json.dumps(TRUST_POLICY)
)
iam.put_role_policy(
    RoleName=IAM_ROLE_NAME,
    PolicyName = POLICY_NAME,
    PolicyDocument = json.dumps(POLICY)
)

roleArn = role['Role']['Arn']

print(f"IAM Role: {roleArn[:13]}{'*' * 12}{roleArn[25:]}")

print("Creating the agent...")

# Create the Bedrock Agent
response = bedrock_agent.create_agent(
    agentName=AGENT_NAME,
    foundationModel=FOUNDATIONAL_MODEL,
    instruction=INSTRUCTION,
    agentResourceRoleArn=roleArn,
)

agentId = response['agent']['agentId']

print("Waiting for agent status of 'NOT_PREPARED'...")
# Wait for agent to reach 'NOT_PREPARED' status
agentStatus = ''
while agentStatus != 'NOT_PREPARED':
    response = bedrock_agent.get_agent(
        agentId=agentId
    )
    agentStatus = response['agent']['agentStatus']
    print(f"Agent status: {agentStatus}")
    time.sleep(2)

######################################### Configure code interpreter for the agent
response = bedrock_agent.create_agent_action_group(
    
    actionGroupName='CodeInterpreterAction',
    actionGroupState='ENABLED',
    agentId=agentId,
    agentVersion='DRAFT',
    parentActionGroupSignature='AMAZON.CodeInterpreter' # <-  To allow your agent to generate, 
                                                        #     run, and troubleshoot code when trying 
                                                        #     to complete a task, set this field to 
                                                        #     AMAZON.CodeInterpreter. 
                                                        #     You must leave the `description`, `apiSchema`, 
                                                        #     and `actionGroupExecutor` fields blank for 
                                                        #     this action group.
)

actionGroupId = response['agentActionGroup']['actionGroupId']

print("Waiting for action group status of 'ENABLED'...")

# Wait for action group to reach 'ENABLED' status
actionGroupStatus = ''
while actionGroupStatus != 'ENABLED':
    response = bedrock_agent.get_agent_action_group(
        agentId=agentId,
        actionGroupId=actionGroupId,
        agentVersion='DRAFT'
    )
    actionGroupStatus = response['agentActionGroup']['actionGroupState']
    print(f"Action Group status: {actionGroupStatus}")
    time.sleep(2)

print("Preparing the agent...")

# Prepare the agent for use
response = bedrock_agent.prepare_agent(
    agentId=agentId
)

print("Waiting for agent status of 'PREPARED'...")

# Wait for agent to reach 'PREPARED' status
agentStatus = ''
while agentStatus != 'PREPARED':
    response = bedrock_agent.get_agent(
        agentId=agentId
    )
    agentStatus = response['agent']['agentStatus']
    print(f"Agent status: {agentStatus}")
    time.sleep(2)

# Create an alias for the agent
response = bedrock_agent.create_agent_alias(
    agentAliasName='test',
    agentId=agentId
)

agentAliasId = response['agentAlias']['agentAliasId']

# Wait for agent alias to be prepared
agentAliasStatus = ''
while agentAliasStatus != 'PREPARED':
    response = bedrock_agent.get_agent_alias(
        agentId=agentId,
        agentAliasId=agentAliasId
    )
    agentAliasStatus = response['agentAlias']['agentAliasStatus']
    print(f"Agent alias status: {agentAliasStatus}")
    time.sleep(2)

print('Done.\n')

print(f"agentId: {agentId}, agentAliasId: {agentAliasId}")

## 2) Utility Functions

These functions are useful to use when interacting with the agent.

In [ ]:
BEDROCK_AGENT_RUNTIME = boto3.client(service_name = 'bedrock-agent-runtime', region_name = REGION_NAME)

In [ ]:
def read_csv_bytes(file_path):
    """Reads a CSV file and returns its content as bytes.

    Args:
        file_path (str): The path to the CSV file.

    Returns:
        bytes: The content of the CSV file as bytes, or None if an error occurs.
    """
    try:
        with open(file_path, 'rb') as file:
            csv_bytes = file.read()
            return csv_bytes
    except FileNotFoundError:
        print(f"Error: File not found: {file_path}")
        return None
    except Exception as e:
         print(f"An error occurred: {e}")
         return None

def invoke(inputText, sessionId, file_path, showTrace=False, endSession=False):

    try:

        # Invoke the Agent - Sends a prompt for the agent to process and respond to.
        response = BEDROCK_AGENT_RUNTIME.invoke_agent(
            agentAliasId=agentAliasId,   # (string) – [REQUIRED] The alias of the agent to use.
            agentId=agentId,             # (string) – [REQUIRED] The unique identifier of the agent to use.
            sessionId=sessionId,         # (string) – [REQUIRED] The unique identifier of the session. Use the same value across requests to continue the same conversation.
            inputText=inputText,         # (string) - The prompt text to send the agent.
            endSession=endSession,       # (boolean) – Specifies whether to end the session with the agent or not.
            enableTrace=True,            # (boolean) – Specifies whether to turn on the trace or not to track the agent's reasoning process.
            sessionState = {
                "files": [
                    {
                        "name": "test_data.json",
                        "source": {
                            "byteContent": {
                                "data": read_csv_bytes(file_path=file_path),
                                "mediaType": "text/csv"
                            },
                            "sourceType": "BYTE_CONTENT"
                       },
                        "useCase": "CODE_INTERPRETER"
                    }
                ]
             }
        )

        # The response of this operation contains an EventStream member. 
        event_stream = response["completion"]

        # When iterated the EventStream will yield events.
        for event in event_stream:

            # chunk contains a part of an agent response
            if 'chunk' in event:
                chunk = event['chunk']
                if 'bytes' in chunk:
                    text = chunk['bytes'].decode('utf-8')
                    print(f"Chunk: {text}")
                else:
                    print("Chunk doesn't contain 'bytes'")

            # files contains intermediate response for code interpreter if any files have been generated.
            if 'files' in event:
                files = event['files']['files']
                for file in files:
                    name = file['name']
                    type = file['type']
                    bytes_data = file['bytes']
                    
                    # It the file is a PNG image then we can display it...
                    if type == 'image/png':
                        # Display PNG image using Matplotlib
                        img = plt.imread(io.BytesIO(bytes_data))
                        plt.figure(figsize=(10, 10))
                        plt.imshow(img)
                        plt.axis('off')
                        plt.title(name)
                        plt.show()
                        plt.close()
                        
                    # If the file is NOT a PNG then we save it to disk...
                    else:
                        # Save other file types to local disk
                        with open(name, 'wb') as f:
                            f.write(bytes_data)
                        print(f"File '{name}' saved to disk.")

    except Exception as e:
        print(f"Error: {e}")

In [3]:
BEDROCK_AGENT_RUNTIME = boto3.client(service_name = 'bedrock-agent-runtime', region_name = "us-west-2")

In [6]:
BEDROCK_AGENT_RUNTIME.list_sessions(maxResults=25)

{'ResponseMetadata': {'RequestId': '1a7ec54c-f496-4f57-82b2-0b17e00f38b8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 25 Mar 2025 23:53:04 GMT',
   'content-type': 'application/json',
   'content-length': '23',
   'connection': 'keep-alive',
   'x-amzn-requestid': '1a7ec54c-f496-4f57-82b2-0b17e00f38b8'},
  'RetryAttempts': 0},
 'sessionSummaries': []}